In [ ]:
import pandas as pd
import numpy as np

def calculate_robust_accuracy(truth_df, test_df, threshold=0.01):
    # Perform an outer join to align rows by index (or a specific key column)
    merged_df = truth_df.merge(test_df, how='outer', left_index=True, right_index=True, suffixes=('_truth', '_test'))

    # Identify rows that are missing in either dataframe
    missing_in_truth = merged_df.filter(regex='_test').isna().any(axis=1)
    missing_in_test = merged_df.filter(regex='_truth').isna().any(axis=1)

    # Calculate element-wise absolute differences for non-missing rows
    diff_df = np.abs(merged_df.filter(regex='_truth').values - merged_df.filter(regex='_test').values)

    # Handle missing rows: Imputation or Penalization
    # Here, we simply penalize missing rows by adding them as 100% error
    missing_penalty = missing_in_truth.sum() + missing_in_test.sum()

    # Determine if differences are within the threshold
    within_threshold = (diff_df <= threshold).sum(axis=1)

    # Calculate accuracy excluding penalties
    accuracy = within_threshold.sum() / merged_df.shape[0]  # Mean over all non-missing elements

    # Adjust accuracy for missing rows
    adjusted_accuracy = accuracy - (missing_penalty / merged_df.shape[0])

    return max(adjusted_accuracy, 0)  # Ensure accuracy is not negative

# Example usage
# truth_df = pd.DataFrame(...)  # Your truth dataframe
# test_df = pd.DataFrame(...)   # Your test dataframe
# accuracy = calculate_robust_accuracy(truth_df, test_df, threshold=0.01)
# print(f"Adjusted Accuracy: {accuracy * 100:.2f}%")
